In [197]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from lxml import html
from datetime import datetime
from pymongo import MongoClient

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get('https://news.mail.ru/')

r = driver.page_source

root = html.fromstring(r)

news = []

main_news = root.xpath('//div[contains (@class, "daynews__item")]')
for newsblock in main_news:
    title = newsblock.xpath('.//span[contains(@class, "photo__title")]/text()')[0].replace(u'\xa0', u' ')
    link = newsblock.xpath('.//a[contains(@class, "photo")]/@href')[0]
    
    #на мейле нет поля с датой, но основные новости опубликованы на текущий день
    date =  datetime.today().strftime('%d-%m-%Y')
    
    news.append({'title':title, 'link':link, 'date':date})

mongo = MongoClient('mongodb://localhost:27017')

db = mongo['gbru']
collection = db['news']

def update_news():
    added = updated = 0
    for entry in news:
        cur_link = entry['link']
        if not collection.find_one({'link':cur_link}):
            collection.insert_one(entry)
            added+=1
        else:
            collection.update_one({'link': cur_link}, {'$set': entry})
            updated+=1
    return {'added':added, 'updated':updated}

print(update_news())



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Rustam\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


{'added': 0, 'updated': 5}
